# ETL-pipelines in Azure Databricks
## Module 4: Data Transformatie en Cleaning
In deze module leren we hoe we declaratiegegevens kunnen opschonen en transformeren in Apache Spark.

We gebruiken hiervoor **twee datasets**:
- `declaraties_dataset.csv` (bevat declaraties per verzekerde)
- `verzekerden_dataset.csv` (bevat gegevens over de verzekerden)


## Stap 1: Start een SparkSession
We beginnen met het opzetten van een SparkSession, de kern van Apache Spark.

In [ ]:
from pyspark.sql import SparkSession

# Start een Spark sessie
spark = SparkSession.builder.appName("ETL_Declaraties").getOrCreate()

## Stap 2: Laad de datasets met declaraties en verzekerden
We laden de datasets en verkennen de gegevens.

In [ ]:
# Laad declaraties dataset
df_declaraties = spark.read.csv("/mnt/data/declaraties_dataset.csv", header=True, inferSchema=True)
df_verzekerden = spark.read.csv("/mnt/data/verzekerden_dataset.csv", header=True, inferSchema=True)

# Toon de eerste records van beide datasets
df_declaraties.show(5)
df_verzekerden.show(5)

## Stap 3: Controleer en behandel ontbrekende waarden
Soms missen er gegevens. We verwijderen of vullen deze aan.

In [ ]:
from pyspark.sql.functions import col, mean

# Toon aantal ontbrekende waarden per kolom
df_declaraties.select([col(c).isNull().sum().alias(c) for c in df_declaraties.columns]).show()

# Vul missende declaratiebedragen met het gemiddelde
avg_bedrag = df_declaraties.select(mean(col("Bedrag"))).collect()[0][0]
df_clean = df_declaraties.fillna({"Bedrag": avg_bedrag})

## Stap 4: Filter onrealistische declaratiebedragen
We behouden alleen bedragen tussen €10 en €500.

In [ ]:
df_filtered = df_clean.filter((col("Bedrag") >= 10) & (col("Bedrag") <= 500))

## Stap 5: Verwijder dubbele declaraties
Dubbele records worden verwijderd op basis van Declaratie_ID.

In [ ]:
df_deduplicated = df_filtered.dropDuplicates(["Declaratie_ID"])

## Stap 6: Groepeer declaraties per zorgtype
We berekenen de totale kosten per zorgtype.

In [ ]:
df_grouped = df_deduplicated.groupBy("Zorgtype").sum("Bedrag")
df_grouped.show()

## Stap 7: JOIN met verzekerdeninformatie
We combineren declaratiegegevens met verzekerdeninformatie.

In [ ]:
# JOIN declaraties met verzekerden op Verzekerde_ID
df_combined = df_deduplicated.join(df_verzekerden, "Verzekerde_ID", "inner")
df_combined.show(5)

## Stap 8: Filter alleen goedgekeurde declaraties
We houden alleen declaraties over met de status 'Goedgekeurd'.

In [ ]:
df_approved = df_combined.filter(col("Declaratie_Status") == "Goedgekeurd")
df_approved.show(5)

## Stap 9: Opslaan in Delta Lake
De opgeschoonde dataset wordt opgeslagen in Delta Lake.

In [ ]:
df_approved.write.format("delta").mode("overwrite").save("/mnt/data/declaraties_opgeschoond")

## Oefeningen
1. **Tel het aantal declaraties per verzekerde en toon alleen de top 10 verzekerden met de meeste declaraties.**
2. **Bereken het totaalbedrag aan declaraties per maand.**